In [1]:
n_scenario = 16

num_scenario = '/' + str(n_scenario) + '_Scenario/'
dir_name = 'sm_' + str(n_scenario) +'/'

import os
import yaml
import json
import shutil
import numpy as np
from gurobipy import GRB
from utils import prepare_input
from main_model import two_stage_model

In [ ]:
with open(r'config.yaml') as file:
    model_params = yaml.load(file, Loader=yaml.FullLoader)

model_params["path_to_input"] = os.path.dirname(os.getcwd()) + '/Data/fixed_reduced_grid' + num_scenario
model_params["path_to_output"] = os.getcwd() + "/output/" + dir_name

if os.path.exists(model_params["path_to_output"]):
    shutil.rmtree(model_params["path_to_output"], ignore_errors=True)
os.mkdir(model_params["path_to_output"])
model_params["input1"], model_params["input2"] = prepare_input(model_params["path_to_input"])

print("MIP-Gap: ", model_params["mip_gap"])
print("Time Limit: ", model_params["time_limit"])
print("Robust Model: ", model_params["robust_flag"])
print("Flexible Generation: ", model_params["flexible_generation"])
print("Objective Type:\t", model_params["set_objective"], "\n")
print("Solver Method:\t", model_params["solver_method"], "\n")

## Changes

In [ ]:
%%time
budget_vector = [0,10,20,30,40,50,60,70,80,85.625]
base_model = two_stage_model(model_params)

### Model Run

In [ ]:
for budget in budget_vector:
    base_model.budget_ref.rhs = budget*1e6
    base_model.model.setParam("LogFile", model_params["path_to_output"] + str(budget) + "M")
    base_model.model.setParam("MIPGap", model_params["mip_gap"])
    base_model.model.setParam("TimeLimit", model_params["time_limit"])
    base_model.model.setParam("Method", model_params["solver_method"])
    base_model.model.optimize()
    base_model.model.write(model_params["path_to_output"] + str(budget) + "M_" + "solution.sol")    
    print("\n\n*********************************************")
    print("*********************************************\n\n")
    
    # budget_solution[budget]["objective_value"] = base_model.model.ObjVal
    # for i in base_model.x.keys():
    #     budget_solution[budget]['substation_' + str(i)] = base_model.x[i].X

### Write Solutions

In [ ]:
with open(model_params["path_to_output"] + 'model_params.json', 'w') as fp:
    del model_params["input1"]
    del model_params["input2"]
    json.dump(model_params, fp)
    
# with open(model_params["path_to_output"] + 'budget_solution.json', 'w') as fp:
#     json.dump(budget_solution, fp)

### Minimum budget needed to satisfy entire demand

In [2]:
with open(r'config.yaml') as file:
    model_params = yaml.load(file, Loader=yaml.FullLoader)

model_params["path_to_input"] = os.path.dirname(os.getcwd()) + '/Data/fixed_reduced_grid' + num_scenario
model_params["path_to_output"] = os.getcwd() + "/output/" + dir_name

model_params["input1"], model_params["input2"] = prepare_input(model_params["path_to_input"])

In [3]:
# For computing zero load shed for mean scenario

model_params["mip_gap"] = 0
# mean_scenario = np.ceil(model_params["input1"].iloc[:,9:].mean(axis=1))

# for i in model_params["input1"].columns:
#     if "max_flood" in i:
#         model_params["input1"].drop(i, axis=1, inplace=True)

# model_params["input1"]["max_flood"] = mean_scenario

In [4]:
%%time

base_model = two_stage_model(model_params)
print("Number of Scenarios:", base_model.n_scenarios)

# Turn the budget constraint from hard to soft
base_model.model.remove(base_model.budget_ref)
base_model.model.setObjective(base_model.fc*base_model.y.sum() + 
                              model_params["mit_coarse"]*base_model.vc*base_model.x.sum(), GRB.MINIMIZE)

# Turn demand satisified constraint from inequality to equality
base_model.model.remove(base_model.supply_demand_constraint_reference)
base_model.model.addConstrs(base_model.s[i,j] == base_model.input1.values[i,8] 
                            for i in range(base_model.n_buses) for j in range(base_model.n_scenarios))

# Model run parameters
base_model.model.setParam("MIPGap", model_params["mip_gap"])
base_model.model.setParam("TimeLimit", model_params["time_limit"])
base_model.model.setParam("Method", model_params["solver_method"])

print("MIP-Gap: ", model_params["mip_gap"])
print("Time Limit: ", model_params["time_limit"])
print("Robust Model: ", model_params["robust_flag"])
print("Flexible Generation: ", model_params["flexible_generation"])
print("Objective Type:\t", model_params["set_objective"], "\n")

base_model.model.optimize()


Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-11-26
Number of Scenarios: 16
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 21600
Set parameter Method to value 2
MIP-Gap:  0
Time Limit:  21600
Robust Model:  False
Flexible Generation:  True
Objective Type:	 min 

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 211802 rows, 77908 columns and 583748 nonzeros
Model fingerprint: 0x713b8915
Variable types: 55968 continuous, 21940 integer (21578 binary)
Coefficient statistics:
  Matrix range     [4e-03, 9e+03]
  Objective range  [2e+04, 1e+05]
  Bounds range     [1e+00, 3e+00]
  RHS range        [1e-03, 2e+04]
Presolve removed 190918 rows and 59934 columns
Presolve time: 2.36s
Presolved: 20884 rows, 17974 columns, 80816 nonzeros
V

In [ ]:
counter = 0

for i in model_params["input1"]["SubNum"].unique():
    if base_model.x[i].X > 0:
        counter = counter + 1

In [ ]:
counter